# Importing Data

Importing training data

In [ ]:
import pandas as pd
df=pd.read_csv('/home/ritwik/Downloads/capston_ project_task/dataset/Subtask-A-master/V1.4_Training.csv',header=None)

Importing testing data

In [ ]:
df_test=pd.read_csv('/home/ritwik/Downloads/capston_ project_task/dataset/Subtask-A-master/SubtaskA_Trial_Test_Labeled.csv',encoding="latin-1")

# Importing Libraries

In [ ]:
import num2words
import nltk
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
from nltk.stem.porter import *
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
stemmer=PorterStemmer()
from gensim.models import Word2Vec
from gensim.models import FastText


### Renaming the columns

In [ ]:
df=df.rename(columns={0:'serial',1:'Sentence',2:'Suggestion_or_not'})

# Preprocessing

Function to preprocess text by lemmatizing the words and by removing the stopwords

In [ ]:
def preprocess_text(text):
    lemmatizer=WordNetLemmatizer()
    text=text.lower()
    token1=tokenizer.tokenize(text)
    token=[]
    for x in token1:
        if x not in stop_words:
            token.append(x)
            #stemmed = [stemmer.stem(tokens) ]
    lemmatiz=[lemmatizer.lemmatize(tokens) for tokens in token]
    return lemmatiz

Creating Data in the paragraph form for applying word2vec model

In [ ]:
para=[[] for i in range(len(df))]
for i in range(len(df)):
    text=df.iloc[i][1]
    text=str(text)
    para[i]=preprocess_text(text)

# Applying Word2Vec

In [ ]:
model_para=Word2Vec(para, min_count=1)
model_para.init_sims(replace=True)

# Similarity calculation using Word2vec and wmdistance

This function calculates the similarity of each sentence in the training data with each sentence in the testing data and assigns the sentence to the class of the document having the maximum similarity

In [ ]:
import math
def get_answers():
    prediction=[]
    for j in range(1,100):
        sent=preprocess_text(df_test.iloc[j][1])
        count=0
        min1=math.inf
        for i in range(len(df)):
            #calculating word mover's distance which calculates the distance between vectors by taking their contexts into account
            distance=model_para.wmdistance(sent,para[i])
            # calculating the mininmum possible distance with any sentence
            if distance<min1:
                min1=distance
                result=df.iloc[i][2]
            count=count+1
        prediction.append(result)
    return prediction

# Getting Predictions

In [ ]:
p=get_answers()

# Getting accuracy

In [ ]:
truelabels=df_test.iloc[:, 2].tolist()

count=0
for i in range(len(p)):
    if p[i]==truelabels[i]:
        count=count+1
print(count/len(p))

# Writing to a csv file

Writing results of top 100 testing files as it is taking a lot of time to run

In [ ]:
with open('Result_Word2Vec', "w") as outfile:
    for entries in p:
        outfile.write(str(entries))
        outfile.write("\n")